In [29]:
import pandas as pd

df = pd.read_csv('../Data/train_ship_segmentations_v2.csv')
df.head()

,ImageId,EncodedPixels
0,00003e153.jpg,NaN
1,0001124c7.jpg,NaN
2,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...
3,000194a2d.jpg,360486 1 361252 4 362019 5 362785 8 363552 10 ...
4,000194a2d.jpg,51834 9 52602 9 53370 9 54138 9 54906 9 55674 ...


In [30]:
df.shape

(231723, 2)

In [32]:
import numpy as np

df['is_empty'] = np.where(df['EncodedPixels'].isna(), 1, 0)

df.head()

,ImageId,EncodedPixels,is_empty
0,00003e153.jpg,NaN,1
1,0001124c7.jpg,NaN,1
2,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...,0
3,000194a2d.jpg,360486 1 361252 4 362019 5 362785 8 363552 10 ...,0
4,000194a2d.jpg,51834 9 52602 9 53370 9 54138 9 54906 9 55674 ...,0


In [33]:
! pip install opencv-python

You are using pip version 10.0.1, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [34]:
! pip install tqdm

You are using pip version 10.0.1, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [44]:
images_with_ship = df.loc[df['is_empty'] == False]
images_with_ship.shape

(81723, 3)

In [ ]:
# Trying to get bounding box coords

# import os
# import cv2
# from tqdm import tqdm
# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# from skimage.io import imread
# import matplotlib.pyplot as plt
# from skimage.segmentation import mark_boundaries
# from skimage.measure import label, regionprops
# from skimage.util.montage import montage2d as montage

# # montage_rgb = lambda x: np.stack([montage(x[:, :, :, i]) for i in range(x.shape[3])], -1)
# # ship_dir = '../input'
# # train_image_dir = 'trainimages'
# # test_image_dir = 'testimages'

# def rle_decode(mask_rle, shape=(768, 768)):
#     '''
#     mask_rle: run-length as string formated (start length)
#     shape: (height,width) of array to return 
#     Returns numpy array, 1 - mask, 0 - background
#     '''
#     s = mask_rle.split()
#     starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
#     starts -= 1
#     ends = starts + lengths
#     img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
#     for lo, hi in zip(starts, ends):
#         img[lo:hi] = 1
#     return img.reshape(shape).T  # Needed to align to RLE direction


# def masks_as_image(in_mask_list, all_masks=None):
#     # Take the individual ship masks and create a single mask array for all ships
#     if all_masks is None:
#         all_masks = np.zeros((768, 768), dtype = np.int16)
#     #if isinstance(in_mask_list, list):
#     for mask in in_mask_list:
#         if isinstance(mask, str):
#             all_masks += rle_decode(mask)
#     return np.expand_dims(all_masks, -1)


import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from skimage.io import imread
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
from skimage.util.montage import montage2d as montage

montage_rgb = lambda x: np.stack([montage(x[:, :, :, i]) for i in range(x.shape[3])], -1)
ship_dir = '../input'
train_image_dir = os.path.join(ship_dir, 'train_v2')
test_image_dir = os.path.join(ship_dir, 'test_v2')

import gc; gc.enable() # memory is tight
from skimage.morphology import label

def multi_rle_encode(img):
    labels = label(img[:, :, 0])
    return [rle_encode(labels==k) for k in np.unique(labels[labels>0])]


def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction


def masks_as_image(in_mask_list):
    # Take the individual ship masks and create a single mask array for all ships
    all_masks = np.zeros((768, 768), dtype = np.int16)
    #if isinstance(in_mask_list, list):
    for mask in in_mask_list:
        if isinstance(mask, str):
            all_masks += rle_decode(mask)
    return np.expand_dims(all_masks, -1)


for i, image in images_with_ship.iterrows():
    if i > 10:
        break
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (15, 5))
    rle_0 = image['EncodedPixels']
    mask_0 = masks_as_image(rle_0)
    #
    # 
    print(image)
    lbl_0 = label(mask_0) 
    props = regionprops(lbl_0)
    for prop in props:
        print('Found bbox', prop.bbox)
#         cv2.rectangle(img_1, (prop.bbox[1], prop.bbox[0]), (prop.bbox[3], prop.bbox[2]), (255, 0, 0), 2)
